In [ ]:
%pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 torchtext==0.14.1 torchaudio==0.13.1 torchdata==0.5.1 --extra-index-url https://download.pytorch.org/whl/cu117 -q

!pip install wandb --quiet
!pip install python-Levenshtein -q
!git clone --recursive https://github.com/parlance/ctcdecode.git
!pip install wget -q
%cd ctcdecode
!pip install . -q
%cd ..

!pip install torchsummaryX -q
!pip install mne -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 GB 835.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.3/24.3 MB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 61.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 97.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.6/248.6 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 18.3 MB/s eta 0:00:00
Cloning into 'ctcdecode'...
remote: Enumerating objects: 1102, done.
remote: Counting objects: 100% (39/39), done.
r

In [ ]:
#!activate pytorch_cuda
import mne

import torch
import random
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torchsummaryX import summary
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

# import torchaudio.transforms as tat

from sklearn.metrics import accuracy_score
import gc

import zipfile
import pandas as pd
from tqdm import tqdm
import os
import datetime

# imports for decoding and distance calculation
import ctcdecode
import Levenshtein
from ctcdecode import CTCBeamDecoder

import warnings
warnings.filterwarnings('ignore')

if torch.backends.mps.is_available():
    device = "mps"
elif torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

print("Device: ", device)

Device:  cuda


In [ ]:
CMUdict_ARPAbet = {
    "" : " ",
    "[SIL]": "-", "NG": "G", "F" : "f", "M" : "m", "AE": "@",
    "R"    : "r", "UW": "u", "N" : "n", "IY": "i", "AW": "W",
    "V"    : "v", "UH": "U", "OW": "o", "AA": "a", "ER": "R",
    "HH"   : "h", "Z" : "z", "K" : "k", "CH": "C", "W" : "w",
    "EY"   : "e", "ZH": "Z", "T" : "t", "EH": "E", "Y" : "y",
    "AH"   : "A", "B" : "b", "P" : "p", "TH": "T", "DH": "D",
    "AO"   : "c", "G" : "g", "L" : "l", "JH": "j", "OY": "O",
    "SH"   : "S", "D" : "d", "AY": "Y", "S" : "s", "IH": "I",
    "[SOS]": "[SOS]", "[EOS]": "[EOS]"
}

CMUdict = list(CMUdict_ARPAbet.keys())
ARPAbet = list(CMUdict_ARPAbet.values())


PHONEMES = CMUdict[:-2]
LABELS = ARPAbet[:-2]

In [ ]:
from google.colab import drive # Link your drive if you are a colab user
drive.mount('/content/drive') # Models in this HW take a long time to get trained and make sure to save it her

Mounted at /content/drive


In [ ]:
# Here, we converted each word into a phoneme using the CMUDict
# This took a rediculous amount of time. Don't rerun this. Instead, load the
# New csv file with phonemes added to it!
wannaSpendForeverMakingDataAgain = False

if wannaSpendForeverMakingDataAgain:
  import pickle

  # # Now its time to make the target labels

  !pip install cmudict -q

  import cmudict

  data_csv =pd.read_csv("drive/MyDrive/brennan data/AliceChapterOne-EEG-Phonemes.csv")
  word_list = data_csv["Word"].values
  phoneme_list = []

  for i, word in enumerate(word_list):
      if i % 25 == 0:
        print(i)
      try:
        phoneme_list.append(cmudict.dict()[word.lower()][-1])
      except:
        # There seems to be weird bug with one of the words???
        # We will need to delete the corresponding timestamp EEG data,
        # so print the index
        print(i, word)

  with open('drive/MyDrive/brennan data/IDL_project_data_targets.pkl', 'wb') as fid:
      pickle.dump(phoneme_list, fid)

In [ ]:
# Load in the new and glorious Phoneme-added dataframe
import pandas as pd
data_csv =pd.read_csv("drive/MyDrive/brennan data/AliceChapterOne-EEG-Phonemes.csv")
data_csv.head()

,Unnamed: 0,Word,Segment,onset,offset,Order,LogFreq,LogFreq_Prev,LogFreq_Next,SndPower,Length,Position,Sentence,IsLexical,NGRAM,RNN,CFG,phonemes
0,0,Alice,1,0.046000,0.608721,1,8.65,0.00,14.56,3.620000e-07,0.562721,1,1,1.0,3.226499,3.126175,2.312348,"['AE1', 'L', 'IH0', 'S']"
1,1,was,1,0.562721,0.830543,2,14.56,8.65,10.69,3.840000e-09,0.267822,2,1,0.0,0.905229,1.691128,1.357460,"['W', 'AH0', 'Z']"
2,2,beginning,1,0.784543,1.302929,3,10.69,14.56,16.35,3.690000e-09,0.518386,3,1,1.0,4.446766,4.100771,5.626722,"['B', 'IH0', 'G', 'IH1', 'N', 'IH0', 'NG']"
3,3,to,1,1.256929,1.398925,4,16.35,10.69,13.79,3.970000e-09,0.141996,4,1,0.0,2.537495,3.833313,5.939201,"['T', 'AH0']"
4,4,get,1,1.352925,1.662327,5,13.79,16.35,13.28,3.770000e-09,0.309402,5,1,0.0,1.023137,1.013076,2.697304,"['G', 'IH1', 'T']"


In [ ]:
# DATA_DIR    = "drive/MyDrive/brennan data/S01" # TODO: Path where you have downloaded the data

# mne_raw = mne.io.read_raw_brainvision(os.path.join(DATA_DIR, "S01.vhdr"),
                                      # misc=['ECG', 'EMG', 'FootPad'],preload=True)

In [ ]:
# You'll notice in the dataframe that they've timestamped each word with regard to
# the segmented file! That was very nice of them. Let's extract the word onset/offset
import numpy as np
segment_onset_offset = []
for seg_val in np.unique(data_csv["Segment"].values):
  seg = data_csv.loc[data_csv.Segment == seg_val]
  onsets = seg["onset"].values
  offsets = seg["offset"].values
  # Append to onset/offset times
  segment_onset_offset.append(list(zip(onsets, offsets)))


In [ ]:
# Now its time to load in the EEG data
# For this, we have to loop through all the subjects (49) and extract
# each word based on the segment and timestamp
import mne
import glob
from collections import Counter

if wannaSpendForeverMakingDataAgain:

  vhdr_files = glob.glob("drive/MyDrive/brennan data/*/*.vhdr")

  vhdr_files

  eeg_data_list = []
  labels_list = []


  def data_sorter(data_string):
      try:
          return int(data_string.split(r"/")[-1])
      except:
          # handle division by zero error
          # leave empty for now
          return 0

  for vhdr_file in vhdr_files[26:]:

    # Step 1: Load the file
    mne_raw = mne.io.read_raw_brainvision(vhdr_file, misc=['ECG', 'EMG', 'FootPad'],preload=True)


    # A couple of the subjects have a different number of channels.
    # As someone who does EEG data research, I get it.
    # As someone doing this class project, I hate them.
    # Let's just ignore those people with different cap sizes to simplify things
    if mne_raw.get_data().shape[0] == 62:

      # Step 2: Preprocess the filter
      # Note: This is very minimal, but we might not want to do too much more

      # Bandpass filter 0.5 - 40 Hz
      mne_raw = mne_raw.filter(0.5, 40);

      # Common Average Reference
      mne_raw.set_eeg_reference('average', projection=False);

      # Resampling to 100 Hz is done AFTER segmentation into words to not lose
      # time resolution when cropping (see: .resample(100) below)

      # Step 3: Segment into the 12 time-stamped segments
      # For some reason, some of the dataset first stimulus is called "stimulus 1"
      # and other times is new segment -.- . Also, calling events from annotations
      # can return the list in unsorted order, which will completely mess up the
      # whole labels to eeg alignment we have going on.
      # So,
      # mne_events, ev_id = mne.events_from_annotations(
      #           raw=mne_raw, event_id={"Stimulus/{:d}".format(i): i for i in range(0,13)})

      mne_events, ev_id = mne.events_from_annotations(raw=mne_raw, event_id=Counter(mne_raw.annotations.description))
      # because strings are weird, this actually returns the events out of order.
      # so, let's sort them according to their timestamp. Easy peasy!
      mne_events = sorted(mne_events, key = lambda x: x[0])

      if len(mne_events) > 12:
        # Only keep last 12 if there was an extra starting stimulus,
        # Otherwise, consider the starting stimulus as start of sequence
        mne_events = mne_events[-12:]

      # We're just going to crop at onset time. This is because the eeg timestamps sometimes overlap
      # into the next segment. *eye roll*
      segments = [mne_raw.copy().crop(mne_events[i][0] * 1/mne_raw.info['sfreq'], None) for i in range(len(mne_events))]

      # Step 4: Further break down each segment into the individual words
      for i in range(len(segments)): # there should be 12extracted_sentences = [sentences[0].copy().crop(timestamps[i], timestamps[i+1]) for i in range(len(timestamps) - 1)]
        #print("Segment: ", i)
        extracted_words = [segments[i].copy().crop(segment_onset_offset[i][j][0], segment_onset_offset[i][j][1]).resample(100) for j in range(len(segment_onset_offset[i]))]
        eeg_data_list.extend(extracted_words)
        #print(eeg_data_list)


In [ ]:
# All this preprocessing is taking 5ever. Let's not do this again.
if not wannaSpendForeverMakingDataAgain:
  import pickle

  # copy the target phoneme list for n subjects
  #targets = phoneme_list * len(vhdr_files) # Repeat the labels for each subject

  with open('drive/MyDrive/brennan data/IDL_project_data_1stHalf.pkl', 'rb') as fid:
      eeg_data_list = pickle.load(fid)

  with open('drive/MyDrive/brennan data/IDL_project_data_targets.pkl', 'rb') as fid:
      phoneme_list = pickle.load(fid)

In [ ]:

#targets = data_csv['phonemes']
#phoneme_list = phoneme_list * len(vhdr_files) # Repeat the labels for each subject

num_subj = int(len(eeg_data_list)/len(phoneme_list))
targets = [phoneme for phoneme in phoneme_list for _ in range(num_subj)]
print(len(targets))


50544


In [ ]:
eeg_data_list[0].get_data().T.shape

(56, 62)

In [ ]:
# split into training, testing, and validation set
import sklearn
from sklearn import model_selection

train_val_targets, test_targets, train_val_eeg, test_eeg = sklearn.model_selection.train_test_split(targets, eeg_data_list, test_size=0.20, random_state=42) # 80/20 train-val/test split
train_targets, val_targets, train_eeg, val_eeg = sklearn.model_selection.train_test_split(train_val_targets, train_val_eeg, test_size=0.20, random_state=42) # 80/20 train / val split

In [ ]:
import torch

class Dataset(torch.utils.data.Dataset):

    def __init__(self, input_data, # EEG data
                 output_data = None, # phoneme data
                 mode = 'Training',
                 phonemes = PHONEMES,
                 min_length = 0,
                ): # Feel free to add more arguments

        """
        Input:
        input_data: Preprocessed EEG data of array of shape (N_samples, ), where each sample is shape (n_time, n_channels),
                    where n_channels is a consistent number across all samples
        output_data: phoneme data of read transcript shape (N_samples, ), where each sample is shape (n_phonemes)

        """
        assert mode in ['Training', 'Testing', 'Validation'], "Error! Unrecognized Mode"

        if mode != "Testing":
          data_tuple = [(i, j) for i,j in zip(input_data, output_data) if i.shape[0] > min_length]
          input_data = [i[0] for i in data_tuple]
          output_data = [i[1] for i in data_tuple]
          print("Input data len: ", len(input_data))
          assert len(input_data) == len(output_data)

        else:
          input_data = [i for i in input_data if i.shape[0] > min_length]


        self.EEG = input_data
        self.length = len(self.EEG)
        self.mode = mode

        if mode != 'Testing':
            self.transcripts = []
            for transcript in output_data: #transcript is one word
                # print(transcript)
                transcript_list = []
                for phoneme in transcript:
                  phoneme = ''.join([i for i in phoneme if not i.isdigit()])
                  transcript_list.append(phonemes.index(phoneme))

                self.transcripts.append(transcript_list)
        #print(self.transcripts)

    def __len__(self):
        return self.length

    def __getitem__(self, ind):

        frames = self.EEG[int(ind)]
        frames = torch.FloatTensor(frames) # Convert to tensors

        if self.mode in ["Training", "Validation"]:
            #labels = torch.tensor(self.transcripts, dtype=torch.long)
            labels    = torch.tensor(self.transcripts[ind], dtype=torch.long)
            return frames, labels

        else:
            return frames

    def collate_fn(self, batch):
        '''
        TODO:
        1.  Extract the features and labels from 'batch'
        2.  We will additionally need to pad both features and labels,
            look at pytorch's docs for pad_sequence
        3.  This is a good place to perform transforms, if you so wish.
            Performing them on batches will speed the process up a bit.
        4.  Return batch of features, labels, lenghts of features,
            and lengths of labels.
        '''
        # batch of input mfcc coefficients
        if self.mode in ["Training", "Validation"]:
            batch_EEG =  [x[0] for x in batch]
            lengths_EEG = [len(x[0]) for x in batch]# TODO
            batch_EEG_pad = pad_sequence(batch_EEG, batch_first=True, padding_value=0.0)

            #batch_transcript = [x[0][1] for x in batch] # TODO
            batch_transcript = [x[1] for x in batch]

            lengths_transcript = [len(x[1]) for x in batch] # TODO
            #print('lengths_transcript is ', lengths_transcript)
            #batch_transcript =  torch.tensor(batch_transcript)


            # Batch x Time x 62 channels
            batch_transcript_pad = pad_sequence(batch_transcript,
                                                batch_first=True,
                                                padding_value=0.0) # TODO

            return batch_EEG_pad, batch_transcript_pad, torch.tensor(lengths_EEG), torch.tensor(lengths_transcript)

        else:
            batch_EEG =  [x for x in batch]
            lengths_EEG = [len(x) for x in batch]# TODO
            batch_EEG_pad = pad_sequence(batch_EEG, batch_first=True, padding_value=0.0)

            return batch_EEG_pad, torch.tensor(batch_EEG)

In [ ]:
config = {
    "beam_width" : 20,
    "lr"         : 2e-3,
    "epochs"     : 50,
    "batch_size" : 256  # Increase if your device can handle it
}

In [ ]:
# Silence the logs because otherwise its just too much

mne.set_log_level(False)

def bandpass_filter_and_stack(mne_raw, all = True):
  if not all:
    theta = mne_raw.copy().filter(4, 8).get_data().T
    alpha = mne_raw.copy().filter(8, 12).get_data().T
    beta = mne_raw.copy().filter(12, 20).get_data().T
    gamma = mne_raw.copy().filter(20, 40).get_data().T

    stacked = np.stack([theta, alpha, beta, gamma], axis = 1)
    return stacked
  else:
    all = mne_raw.copy().filter(0.5, 40).get_data().T
    return all



In [ ]:
train_data = Dataset(input_data = [bandpass_filter_and_stack(i) for i in train_eeg], # .get_data().T for i in range(0, 4)],
                     output_data = train_targets, mode = "Training", min_length = 10)

val_data = Dataset(input_data = [bandpass_filter_and_stack(i) for i in val_eeg], # [extracted_sentences[4].get_data().T],
                     output_data = val_targets, mode = "Validation", min_length = 10)

#test_data = Dataset(input_data = [i.get_data().T for i in test_eeg], #[extracted_sentences[4].get_data().T],
#                    mode = "Testing")

# Do NOT forget to pass in the collate function as parameter while creating the dataloader
train_loader = torch.utils.data.DataLoader(
    dataset     = train_data,
    num_workers = 2, # <--- probably want to increase this :)
    batch_size  = config["batch_size"],
    pin_memory  = True,
    shuffle     = True,
    collate_fn  = train_data.collate_fn
)

val_loader = torch.utils.data.DataLoader(
    dataset     = val_data,
    num_workers = 2,
    batch_size  = config["batch_size"],
    pin_memory  = True,
    shuffle     = False,
    collate_fn  = train_data.collate_fn
)

# test_loader = torch.utils.data.DataLoader(
#     dataset     = test_data,
#     num_workers = 2,
#     batch_size  = 2,  # not a lot of samples . . .
#     pin_memory  = True,
#     shuffle     = False,
#     collate_fn  = train_data.collate_fn
# )

Input data len:  29416
Input data len:  7324


In [ ]:
# sanity check
print("Batch size: ", config["batch_size"])
print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))

for i, data in enumerate(train_loader):
    x, y, lx, ly = data
    print(x.shape, y.shape, lx.shape, ly.shape)
    print(y)
    break



Batch size:  256
Train dataset samples = 29416, batches = 115
torch.Size([256, 83, 62]) torch.Size([256, 10]) torch.Size([256]) torch.Size([256])
tensor([[ 5,  8, 37,  ...,  0,  0,  0],
        [32,  6, 13,  ...,  0,  0,  0],
        [20, 15, 33,  ...,  0,  0,  0],
        ...,
        [15,  0,  0,  ...,  0,  0,  0],
        [20, 12, 37,  ...,  0,  0,  0],
        [24, 33, 39,  ...,  0,  0,  0]])


In [ ]:
padding_value = 0.0

class Network(nn.Module):

    def __init__(self):

        super(Network, self).__init__()
        self.kernel_size = 2
        self.stride = 1

        # Adding some sort of embedding layer or feature extractor might help performance.
        self.embedding = nn.Conv1d(in_channels=62, out_channels=20, kernel_size=self.kernel_size, stride=self.stride)

        # TODO : look up the documentation. You might need to pass some additional parameters.
        self.lstm = nn.LSTM(input_size = 20, hidden_size = 1000, num_layers = 5, bidirectional=True, dropout = 0.0, batch_first=True)

        self.batch_norm = nn.Sequential(nn.BatchNorm1d(2000))

        self.classification = nn.Sequential(
            nn.Linear(2000,512),
            # nn.ReLU(),
            # nn.Dropout(p=0.25),
            #nn.Linear(512,512),
            #nn.Dropout(p=.25),
            #nn.Linear(512,512),
            #nn.Dropout(p=.25),
            #nn.Linear(512,512),
            #nn.Dropout(p=.25),
            #nn.Linear(512,512),
            #nn.Dropout(p=.25),
            #nn.Linear(512,512),
            #nn.Dropout(p=.25),
            nn.Linear(512,41)
            #TODO: Linear layer with in_features from the lstm module above and out_features = OUT_SIZE
        )


        self.logSoftmax = nn.LogSoftmax(dim=2)#TODO: Apply a log softmax here. Which dimension would apply it on ?

    def forward(self, x, lx):
        #TODO
        # The forward function takes 2 parameter inputs here. Why?
        # Refer to the handout for hints
        # print(x)
        # Permute before and after CNN

        # Remove 0 padding
        # Probably need to rewrite code
        # Code taken from: https://stackoverflow.com/a/59076768
        valid_cols = [col_idx for col_idx, col in enumerate(torch.split(x, 1, dim=0)) if not torch.all(col == 1)]
        x = x[:, valid_cols, :]

        x = torch.permute(x, (0, 2, 1))
        x = self.embedding(x)
        x = torch.permute(x, (0, 2, 1))
        # print(x)

        # Output length of conv embedding layer
        lx = 1 + (lx - self.kernel_size)//self.stride

        #print(lx)
        x = pack_padded_sequence(x, lx, batch_first=True, enforce_sorted=False)

        x, (h_t, c_t) = self.lstm(x)
        x, lh = pad_packed_sequence(x, batch_first=True, padding_value=padding_value)

        x = torch.permute(x, (0, 2, 1))
        x = self.batch_norm(x)
        x = torch.permute(x, (0, 2, 1))

        x = self.classification(x)
        probs = self.logSoftmax(x)

        return probs, lh

In [ ]:
torch.cuda.empty_cache()

model = Network().to(device)
summary(model, x.to(device), lx) # x and lx come from the sanity check above :)
model = Network().to(device)

RuntimeError: ignored

In [ ]:
#TODO

criterion = torch.nn.CTCLoss() # Define CTC loss as the criterion. How would the losses be reduced?
# CTC Loss: https://pytorch.org/docs/stable/generated/torch.nn.CTCLoss.html
# Refer to the handout for hints

optimizer =  torch.optim.Adam(model.parameters(), lr=0.001) # What goes in here?

# Declare the decoder. Use the CTC Beam Decoder to decode phonemes
# CTC Beam Decoder Doc: https://github.com/parlance/ctcdecode

# CTC Decoder is broke.

# decoder = ctcdecode.CTCBeamDecoder(
#     LABELS,
#     model_path=None,
#     alpha=0,
#     beta=0,
#     cutoff_top_n=40,
#     cutoff_prob=1.0,
#     beam_width=20,
#     num_processes=4,
#     blank_id=0,
#     log_probs_input=True)

# decoder = BeamSearchDecoder
decoder = ctcdecode.CTCBeamDecoder(labels=LABELS, model_path=None, alpha=0, beta=0, cutoff_top_n=40, cutoff_prob=1, beam_width=10, num_processes=4, blank_id=0, log_probs_input=True) #TODO

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=2)
#scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=.95, last_epoch=-1)


# Mixed Precision, if you need it
scaler = torch.cuda.amp.GradScaler()

In [ ]:
def decode_prediction(output, output_lens, decoder, PHONEME_MAP= LABELS):

    # TODO: look at docs for CTC.decoder and find out what is returned here. Check the shape of output and expected shape in decode.
    beam_results, beam_scores, timesteps, out_seq_len = decoder.decode(output, seq_lens= output_lens) #lengths - list of lengths

    pred_strings                    = []

    for i in range(output_lens.shape[0]):
        #TODO: Create the prediction from the output of decoder.decode. Don't forget to map it using PHONEMES_MAP.
        prediction = list(beam_results[i][0][0:out_seq_len[i][0]].cpu().numpy())
        predicted_string = ''.join([PHONEME_MAP[j] for j in prediction])
        pred_strings.append(predicted_string)

    return pred_strings

def calculate_levenshtein(output, label, output_lens, label_lens, decoder, PHONEME_MAP= LABELS): # y - sequence of integers

    dist            = 0
    batch_size      = label.shape[0]
    #batch_size = len(label[0])

    output = torch.permute(output, (1,0,2))
    pred_strings    = decode_prediction(output, output_lens, decoder, PHONEME_MAP)
    for i in range(batch_size):
        # TODO: Get predicted string and label string for each element in the batch
        pred_string = pred_strings[i]
        # print("pred before shortening: ", pred_string)
        label_string = label[i][0:label_lens[i]]
        #print('label string before replace ', label_string)
        label_string = ''.join([PHONEME_MAP[k] for k in label_string])
        print('predicted string is ', pred_string.replace(" ", ""))
        print('label string is     ', label_string.replace(" ", ""))
        dist += Levenshtein.distance(pred_string.replace(" ", ""), label_string.replace(" ", "")) #" " becomes ""

    # Print the last things for sanity check
    # print('predicted string is ', pred_string.replace(" ", ""))
    # print('label string is     ', label_string.replace(" ", ""))

    dist /= batch_size # TODO: Uncomment this, but think about why we are doing this
    return dist

In [ ]:
# test code to check shapes

model.eval()
for i, data in enumerate(val_loader, 0):
    x, y, lx, ly = data
    x, y = x.to(device), y.to(device)
    h, lh = model(x, lx)
    h = torch.permute(h, (1, 0, 2))
    print(h.shape, y.shape)
    print(lh.shape, lh.shape)
    print('h is ', h)
    print('y is ', y)
    loss = criterion(h, y, lh, ly)
    print(loss)
    #print('ly is ', ly)
    print(h.max())
    print(calculate_levenshtein(h, y, lx, ly, decoder, LABELS))

    break

torch.Size([82, 256, 41]) torch.Size([256, 11])
torch.Size([256]) torch.Size([256])
h is  tensor([[[ -1.1009, -11.2855, -10.0301,  ...,  -9.1059,  -3.0048,  -8.8670],
         [ -1.0960, -11.2954, -10.0358,  ...,  -9.1161,  -3.0071,  -8.8745],
         [ -1.0247, -11.4249, -10.1041,  ...,  -9.2432,  -3.0461,  -8.9659],
         ...,
         [ -1.0628, -11.3543, -10.0669,  ...,  -9.1743,  -3.0243,  -8.9165],
         [ -1.1263, -11.2294, -10.0010,  ...,  -9.0582,  -2.9940,  -8.8221],
         [ -1.0810, -11.3219, -10.0499,  ...,  -9.1425,  -3.0147,  -8.8936]],

        [[ -7.8797, -10.7662, -10.1916,  ...,  -2.5471,  -8.8134,  -2.3021],
         [ -7.8612, -10.7671, -10.1833,  ...,  -2.5473,  -8.8033,  -2.2989],
         [ -7.6283, -10.8001, -10.1074,  ...,  -2.5482,  -8.6963,  -2.2642],
         ...,
         [ -7.7524, -10.7811, -10.1483,  ...,  -2.5471,  -8.7525,  -2.2827],
         [ -7.9991, -10.7606, -10.2308,  ...,  -2.5552,  -8.8773,  -2.3009],
         [ -7.8115, -10.7729, -10

EEG to phonemes

input EEG --> output phoneme

EEG = (12 * N_subjects,) varying length

"labels" = list (12 * N_subjects,) varying length


Follow HW3P2 logic

Padpacked sequence whatever dataloader

In [ ]:
import wandb
wandb.login(key="dd406b9c9216ffc7053f08e284381c34b4b95df6")

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
run = wandb.init(
    name = "alpha_band_filtered", ## Wandb creates random run names if you skip this field
    reinit = True, ### Allows reinitalizing runs when you re-run this cell
    # run_id = ### Insert specific run id here if you want to resume a previous run
    # resume = "must" ### You need this to resume previous runs, but comment out reinit = True when using this
    project = "IDL_group_project", ### Project should be created in your wandb account
    config = config ### Wandb Config for your run
)

wandb: Currently logged in as: jkosnoff (idl-f23-bestteam). Use `wandb login --relogin` to force relogin


In [ ]:
# from tqdm import tqdm

# def train_model(model, train_loader, criterion, optimizer):

#     model.train()
#     batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train')

#     total_loss = 0

#     for i, data in enumerate(train_loader):
#         optimizer.zero_grad()

#         x, y, lx, ly = data
#         x, y = x.to(device), y.to(device)

#         with torch.cuda.amp.autocast():
#             h, lh = model(x, lx)
#             h = torch.permute(h, (1, 0, 2))
#             loss = criterion(h, y, lh, ly)

#         total_loss += loss.item()

#         batch_bar.set_postfix(
#             loss="{:.04f}".format(float(total_loss / (i + 1))),
#             lr="{:.06f}".format(float(optimizer.param_groups[0]['lr'])))

#         batch_bar.update() # Update tqdm bar

#         # Another couple things you need for FP16.
#         scaler.scale(loss).backward() # This is a replacement for loss.backward()
#         scaler.step(optimizer) # This is a replacement for optimizer.step()
#         scaler.update() # This is something added just for FP16

#         del x, y, lx, ly, h, lh, loss
#         torch.cuda.empty_cache()

#     batch_bar.close() # You need this to close the tqdm bar

#     return total_loss / len(train_loader)


# def validate_model(model, val_loader, decoder, phoneme_map= LABELS):

#     model.eval()
#     batch_bar = tqdm(total=len(val_loader), dynamic_ncols=True, position=0, leave=False, desc='Val')

#     total_loss = 0
#     vdist = 0

#     for i, data in enumerate(val_loader):

#         x, y, lx, ly = data
#         x, y = x.to(device), y.to(device)

#         with torch.inference_mode():
#             h, lh = model(x, lx)
#             h = torch.permute(h, (1, 0, 2))
#             loss = criterion(h, y, lh, ly)

#         total_loss += float(loss)
#         vdist += calculate_levenshtein(torch.permute(h, (1, 0, 2)), y, lh, ly, decoder, phoneme_map)

#         batch_bar.set_postfix(loss="{:.04f}".format(float(total_loss / (i + 1))), dist="{:.04f}".format(float(vdist / (i + 1))))

#         batch_bar.update()

#         del x, y, lx, ly, h, lh, loss
#         torch.cuda.empty_cache()

#     batch_bar.close()
#     total_loss = total_loss/len(val_loader)
#     val_dist = vdist/len(val_loader)
#     return total_loss, val_dist

In [ ]:
def save_model(model, optimizer, scheduler, metric, epoch, path):
    torch.save(
        {'model_state_dict'         : model.state_dict(),
         'optimizer_state_dict'     : optimizer.state_dict(),
         'scheduler_state_dict'     : scheduler.state_dict(),
         metric[0]                  : metric[1],
         'epoch'                    : epoch},
         path
    )

def load_model(path, model, metric= 'valid_acc', optimizer= None, scheduler= None):

    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])

    if optimizer != None:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    if scheduler != None:
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])

    epoch   = checkpoint['epoch']
    metric  = checkpoint[metric]

    return [model, optimizer, scheduler, epoch, metric]

In [ ]:
# This is for checkpointing, if you're doing it over multiple sessions

last_epoch_completed = 0
start = last_epoch_completed
end = config["epochs"]
best_lev_dist = float("inf") # if you're restarting from some checkpoint, use what you saw there.
#epoch_model_path = #TODO set the model path( Optional, you can just store best one. Make sure to make the changes below )
best_model_path = 'checkpoint.pth' #TODO set best model path

In [ ]:
from tqdm import tqdm

def train(model, train_loader, criterion, optimizer):

    model.train()
    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train')

    total_loss = 0

    for i, data in enumerate(train_loader):
        optimizer.zero_grad()

        x, y, lx, ly = data
        x, y = x.to(device), y.to(device)

        # with torch.cuda.amp.autocast():
        h, lh = model(x, lx)
        h = torch.permute(h, (1, 0, 2))
        loss = criterion(h, y, lh, ly)

        # print(h, y)
        # print(lh, ly)
        # print(loss)
        total_loss += loss.item() if (not np.isnan(loss.item())) and (not np.isinf(loss.item())) else 10

        batch_bar.set_postfix(
            loss="{:.04f}".format(float(total_loss / (i + 1))),
            lr="{:.06f}".format(float(optimizer.param_groups[0]['lr'])))

        batch_bar.update() # Update tqdm bar

        # Another couple things you need for FP16.
        scaler.scale(loss).backward() # This is a replacement for loss.backward()
        scaler.step(optimizer) # This is a replacement for optimizer.step()

        # Clip gradient ?????
        torch.nn.utils.clip_grad_norm_(model.parameters(), 10)

        scaler.update() # This is something added just for FP16

        del x, y, lx, ly, h, lh, loss
        torch.cuda.empty_cache()

    batch_bar.close() # You need this to close the tqdm bar

    return total_loss / len(train_loader)


def validate(model, val_loader, decoder, phoneme_map= LABELS):

    model.eval()
    batch_bar = tqdm(total=len(val_loader), dynamic_ncols=True, position=0, leave=False, desc='Val')

    total_loss = 0
    vdist = 0

    for i, data in enumerate(val_loader):

        x, y, lx, ly = data
        x, y = x.to(device), y.to(device)

        with torch.inference_mode():
            h, lh = model(x, lx)
            h = torch.permute(h, (1, 0, 2))
            loss = criterion(h, y, lh, ly)

        total_loss += loss.item() if (not np.isnan(loss.item())) and (not np.isinf(loss.item())) else 999
        vdist += calculate_levenshtein(h, y, lh, ly, decoder, phoneme_map)

        batch_bar.set_postfix(loss="{:.04f}".format(float(total_loss / (i + 1))), dist="{:.04f}".format(float(vdist / (i + 1))))

        batch_bar.update()

        del x, y, lx, ly, h, lh, loss
        torch.cuda.empty_cache()

    batch_bar.close()
    total_loss = total_loss/len(val_loader)
    val_dist = vdist/len(val_loader)
    return total_loss, val_dist

In [ ]:
torch.cuda.empty_cache()
gc.collect()
best_lev_dist = 100
#TODO: Please complete the training loop

for epoch in range(0, config['epochs']):

    print("\nEpoch: {}/{}".format(epoch+1, config['epochs']))

    curr_lr = float(optimizer.param_groups[0]['lr'])
    train_loss = train(model, train_loader, criterion, optimizer)
    valid_loss, valid_dist = validate(model, val_loader, decoder)
    scheduler.step(valid_loss) #needs loss to step (plateau)
    #scheduler.step() #does not need loss (exponential)

    print("\tTrain Loss {:.04f}\t Learning Rate {:.07f}".format(train_loss, curr_lr))
    print("\tVal Dist {:.04f}\t Val Loss {:.04f}".format(valid_dist, valid_loss))


    wandb.log({
        'train_loss': train_loss,
        'valid_dist': valid_dist,
        'valid_loss': valid_loss,
        'lr'        : curr_lr
    })

    #save_model(model, optimizer, scheduler, ['valid_dist', valid_dist], epoch, epoch_model_path)
    #wandb.save(epoch_model_path)
    #print("Saved epoch model")

    if valid_dist <= best_lev_dist:
        best_lev_dist = valid_dist
        save_model(model, optimizer, scheduler, ['valid_dist', valid_dist], epoch, best_model_path)
        wandb.save(best_model_path)
        print("Saved best model")
      # You may find it interesting to exlplore Wandb Artifcats to version your models
run.finish()


Epoch: 1/50


	Train Loss 5.3006	 Learning Rate 0.0010000
	Val Dist 2.9656	 Val Loss 38.2241
Saved best model

Epoch: 2/50


	Train Loss 3.9087	 Learning Rate 0.0010000
	Val Dist 2.9568	 Val Loss 38.0138
Saved best model

Epoch: 3/50


	Train Loss 3.7733	 Learning Rate 0.0010000
	Val Dist 3.0157	 Val Loss 38.2178

Epoch: 4/50


	Train Loss 3.7019	 Learning Rate 0.0010000
	Val Dist 2.9568	 Val Loss 37.9573
Saved best model

Epoch: 5/50


...

	Train Loss 3.5994	 Learning Rate 0.0002500
	Val Dist 2.9568	 Val Loss 37.8303
Saved best model

Epoch: 18/50


	Train Loss 3.5841	 Learning Rate 0.0002500
	Val Dist 3.0047	 Val Loss 37.8227

Epoch: 19/50


	Train Loss 3.5692	 Learning Rate 0.0002500
	Val Dist 2.9789	 Val Loss 37.8207

Epoch: 20/50


	Train Loss 3.5671	 Learning Rate 0.0001250
	Val Dist 2.9567	 Val Loss 37.8148


KeyboardInterrupt: ignored